In [ ]:
!pip install gensim spacy nltk

In [ ]:
import gensim
import re
import spacy
import warnings
warnings.filterwarnings("ignore")

# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

## Load AraVec
Download a model from the [AraVec Repository](https://github.com/bakrianoo/aravec), then follow the below steps to load it.

In [ ]:
# Download via terminal commands
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip" 
!unzip "full_grams_cbow_100_twitter.zip"

--2021-11-17 15:07:40--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_cbow_100_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1131904811 (1.1G) [application/zip]
Saving to: ‘full_grams_cbow_100_twitter.zip’

full_grams_cbow_100 100%[===================>]   1.05G  33.7MB/s    in 26s     

2021-11-17 15:08:06 (41.3 MB/s) - ‘full_grams_cbow_100_twitter.zip’ saved [1131904811/1131904811]

Archive:  full_grams_cbow_100_twitter.zip
  inflating: full_grams_cbow_100_twitter.mdl  
  inflating: full_grams_cbow_100_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_grams_cbow_100_twitter.mdl.wv.vectors.npy  


In [ ]:
# load the AraVec model
model = gensim.models.Word2Vec.load("full_grams_cbow_100_twitter.mdl")
print("We've",len(model.wv.index2word),"vocabularies")

We've 1476715 vocabularies


## Export the Word2Vec format + gzip it.

In [ ]:
# make a directory called "spacyModel"
%mkdir spacyModel

mkdir: cannot create directory ‘spacyModel’: File exists


In [ ]:
# export the word2vec fomart to the directory
model.wv.save_word2vec_format("./spacyModel/aravec.txt")

In [ ]:
# using `gzip` to compress the .txt file
!gzip ./spacyModel/aravec.txt

## Initialize the spaCy model using AraVec vectors

- This will create a folder called `/spacy.aravec.model` within your current working directory.
- This step could take several minutes to be completed.

In [ ]:
!python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz

✔ Successfully created model
1476715it [01:07, 21745.38it/s]
✔ Loaded vectors from spacyModel/aravec.txt.gz
✔ Sucessfully compiled vocab
1476903 entries, 1476715 vectors


## Run Your AraVec Spacy Model


In [ ]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

In [ ]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = clean_str(text)
        return self.tokenizer(preprocessed)

In [ ]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

## Test the Model

In [ ]:
# Test your model
nlp("قطة").vector

array([ 0.6214019 ,  2.664876  , -2.4490244 , -0.13141291,  1.0106287 ,
        1.4277642 , -0.6019407 , -0.37155798,  2.2610269 , -0.51503485,
       -1.7400011 ,  1.4599515 ,  1.3110927 ,  0.4506139 ,  1.1511235 ,
       -2.3989084 ,  0.0108205 , -0.93597263,  0.20742278,  2.7626824 ,
       -0.21789424, -2.6269352 , -0.033042  , -2.0458148 ,  1.4766251 ,
       -2.589866  , -1.7341375 , -1.5589778 ,  0.57571614,  4.2727513 ,
        0.02701492,  1.77316   , -1.1816478 ,  0.24516247, -0.04227808,
        0.57215565,  3.2628767 , -1.2422727 ,  1.2351261 ,  1.7213373 ,
       -2.2107098 ,  3.334359  ,  0.835815  ,  0.27691752, -0.61994714,
        2.0607152 , -0.33151346,  2.132865  , -1.1516991 , -0.39679298,
       -2.1682317 ,  1.5982645 , -1.1571178 ,  1.3672193 , -0.81996626,
        0.5634883 ,  0.8571397 ,  1.2602032 ,  1.5811064 , -2.6346667 ,
       -0.21950944, -1.7665412 ,  1.3162723 , -0.9176698 , -0.5075662 ,
       -0.6396452 , -0.57308793,  2.6602883 ,  1.466169  , -0.54

In [ ]:
Khaled = '''
لا يعرف الكثير عن خالد خلال فترة الدعوة للإسلام في مكة. وبعد هجرة الرسول من مكة إلى المدينة المنورة، دارت العديد من المعارك بين المسلمين وقريش. لم يخض خالد غزوة بدر أولى المعارك الكبرى بين الفريقين، والتي وقع فيها شقيقه الوليد أسيرًا في أيدي المسلمين. وذهب خالد وشقيقه هشام لفداء الوليد في يثرب، إلا أنه وبعد فترة قصيرة من فدائه، أسلم الوليد وهرب إلى يثرب.

كانت غزوة أحد أولى معارك خالد في الصراع بين القوتين، والتي تولى فيها قيادة ميمنة القرشيين. لعب خالد دورًا حيويًا لصالح القرشيين، فقد استطاع تحويل دفة المعركة، بعدما استغل خطأ رماة المسلمين، عندما تركوا جبل الرماة لجمع الغنائم بعد تفوق المسلمين في بداية المعركة. انتهز خالد ذلك الخطأ ليلتف حول جبل الرماة ويهاجم بفرسانه مؤخرة جيش المسلمين، مما جعل الدائرة تدور على المسلمين، وتحوّل هزيمة القرشيين إلى نصر.

شارك خالد أيضًا في صفوف الأحزاب في غزوة الخندق، وقد تولى هو وعمرو بن العاص تأمين مؤخرة الجيش في مائتي فارس، خوفًا من أن يتعقبهم المسلمون. كما كان على رأس فرسان قريش الذين أرادوا أن يحولوا بين المسلمين ومكة في غزوة الحديبية.

إسلامه
بينما كان المسلمون في مكة لأداء عمرة القضاء في العام السابع الهجري، وفقًا للاتفاق الذي أبرم في صلح الحديبية، أرسل الرسول إلى الوليد بن الوليد، وسأله عن خالد، قائلاً له: «ما مثل خالد يجهل الإسلام، ولو كان جعل نكايته وحده مع المسلمين على المشركين كان خيرًا له، ولقدمناه على غيره.» أرسل الوليد إلى خالد برسالة يدعوه فيها للإسلام إدراكاً لما فاته. وافق ذلك الأمر هوى خالد، فعرض على صفوان بن أمية ثم على عكرمة بن أبي جهل الانضمام إليه في رحلته إلى يثرب ليعلن إسلامه، إلا أنهما رفضا ذلك. ثم عرض الأمر على عثمان بن طلحة العبدري، فوافقه إلى ذلك. وبينما هما في طريقهما إلى يثرب، التقيا عمرو بن العاص مهاجرًا ليعلن إسلامه، فدخل ثلاثتهم يثرب في صفر عام 8 هـ معلنين إسلامهم، وحينها قال الرسول: "إن مكة قد ألقت إلينا أفلاذ كبدها".

فلما وصل المدينة المنورة، قصّ خالد على أبي بكر رؤيا رآها في منامه، كأنه في بلاد ضيقة مجدبة، فخرج إلى بلاد خضراء واسعة، ففسرها له أبو بكر: "مخرجك الذي هداك الله للإسلام، والضيق الذي كنت فيه من الشرك".

سيف الله المسلول
عنت
معارك خالد بن الوليد
Crystal Clear app kdict.png مقالة مفصلة: غزوة مؤتة
في عام 8 هـ، وجّه الرسول جيشًا لقتال الغساسنة، بعد أن اعترض شرحبيل بن عمرو الغساني عامل قيصر الروم على البلقاء الحارث بن عمير الأزدي رسول الرسول محمد إلى صاحب بصرى، وقتله. انضم خالد حديث العهد بالإسلام إلى ذلك الجيش ذي الثلاث آلاف مقاتل. اختار النبي زيد بن حارثة لقيادة الجيش، على أن يخلفه جعفر بن أبي طالب إن قتل، ثم عبد الله بن رواحة إن قتل جعفر، وإن قتل الثلاثة يختار المسلمون قائدًا من بينهم.

عند وصول الجيش إلى مؤتة، وجد المسلمون أنفسهم أمام جيش من مائتي ألف مقاتل نصفهم من الروم والنصف الآخر من الغساسنة. فوجئ المسلمون بالموقف، وأقاموا لليلتين في معان يتشاورون أمرهم. أشار البعض بأن يرسلوا للرسول ليشرحوا له الموقف، وينتظروا إما المدد أو الأوامر الجديدة. عارض ابن رواحة ذلك، وأقنع المسلمين بالقتال. بدأت المعركة، وواجه المسلمون موقفًا عصيبًا، حيث قتل القادة الثلاثة على التوالي، عندئذ اختار المسلمون خالدًا ليقودهم في المعركة. صمد الجيش بقية اليوم، وفي الليل نقل خالد ميمنة جيشه إلى الميسرة، والميسرة إلى الميمنة، وجعل مقدمته موضع الساقة، والساقة موضع المقدمة. ثم أمر طائفة بأن تثير الغبار ويكثرون الجلبة خلف الجيش حتى الصباح. وفي الصباح، فوجئ جيش الروم والغساسنة بتغيّر الوجوه والأعلام عن تلك التي واجهوها بالأمس، إضافة إلى الجلبة، فظنوا أن مددًا قد جاء للمسلمين. عندئذ أمر بالانسحاب وخشي الروم أن يلاحقوهم، خوفًا من أن يكون الانسحاب مكيدة. وبذلك، نجح خالد في أن يحفظ الجيش من إبادة شاملة. حارب خالد ببسالة في غزوة مؤتة، وكسرت في يده يومئذ تسعة أسياف. وبعد أن عاد إلى يثرب، أثنى عليه الرسول ولقّبه بسيف الله المسلول.

وبعد شهور، نقضت قريش أحد شروط الصلح، عندما هاجم بكر بن مناة بن كنانة حلفاء قريش بني خزاعة حلفاء الرسول. عندئذ توجه الرسول في جيش من عشرة آلاف مقاتل إلى مكة، وقسم الجيش إلى أربعة أقسام تولى بنفسه قيادة أحدها وأمّر الزبير بن العوام وسعد بن عبادة وخالد بن الوليد على الثلاثة الأخرى، وأمرهم أن يدخلوا مكة كلٌ من باب. فدخلوها كل من الباب الموكل إليه، ولم يلق أحدهم قتالاً إلا كتيبة خالد، حيث قاتله عكرمة بن أبي جهل وسهيل بن عمرو وصفوان بن أمية في جند جمعوه لقتال المسلمين، واستطاع خالد أن يظفر بهم، وقتل منهم عددًا. ثم أرسله الرسول في سرية من ثلاثين فارسًا لهدم العزى صنم جميع بني كنانة، فهدمها ثم رجع إلى الرسول، فأخبره فسأله الرسول إن كان قد رأى شيئًا، فرد بالنفي، فطلب منه الرسول أن يعود لأنه لم يهدمها. فرجع خالد وهو متغيظ فجرد سيفه، فخرجت إليه امرأة عريانة سوداء ناشرة الرأس، فضربها خالد فشقها نصفين ورجع إلى الرسول. فأخبره فقال: "نعم تلك العزى، وقد يئست أن تعبد ببلادكم أبدا!". وكان موضع العزى ومقر عبادتها في شِعب يعرف بشعب سُقام إلى الشمال الشرقي من مكة اتخذه المشركون حمىً لها.

سرية خالد بن الوليد إلى بني جذيمة
بعد الفتح، أرسل الرسول السرايا لدعوة القبائل إلى الإسلام، فأرسل خالد بن الوليد قائدًا على 350 من المهاجرين والأنصار وبني سليم في سرية إلى "بني جذيمة بن عامر بن عبد مناة بن كنانة"، ولم يأمره بقتال. وهنا كانت أول زلاّت خالد حيث قاتلهم، وأصاب منهم، رغم معارضة من كان معه من الصحابة، ومنه سالم مولى أبي حذيفة وعبد الله بن عمر بن الخطاب، فلما وصل الخبر إلى الرسول رفع يديه إلى السماء ثم قال: "اللهم إني أبرأ إليك مما صنع خالد بن الوليد". وأرسل الرسول عليًا إلى بني جذيمة، لدفع ديّة قتلاهم.

ورغم هذا الخطإ، أشركه الرسول بعد ذلك في غزوة حنين، حيث جعله الرسول يومئذ قائدًا على بني سليم، وأصيب يومها إصابات بليغة. كما شارك خالد أيضًا في غزوة تبوك تحت قيادة الرسول، ومن هناك أرسله الرسول في سرية إلى دومة الجندل، فدخلها وأسر صاحبها أكيدر بن عبد الملك الذي صالحه الرسول على الجزية، وهدم صنمهم "وُدّ". في عام 10 هـ، بعث الرسول خالد بن الوليد في شهر ربيع الأول في سرية من أربعمائة مقاتل إلى بني الحارث بن كعب بنجران، وأمره أن يدعوهم إلى الإسلام قبل أن يقاتلهم ثلاثًا، فإن استجابوا له يقبل منهم ويقيم فيهم ويعلمهم دينهم، وإن لم يفعلوا يقاتلهم. لبّى بنو الحارث بن كعب النداء وأسلموا، فأقام خالد فيهم يعلمهم الإسلام. ثم كتب خالد إلى الرسول بذلك، فأمره أن يقيم فيهم يعلمهم، ثم ليقبل معه وفدهم، فوفدوا عليه يعلنون إسلامهم.

خالد في عهد أبي بكر وعمر
حروب الردّة
Crystal Clear app kdict.png مقالة مفصلة: حروب الردة
بنو أسد وطليحة بن خويلد
بعد وفاة الرسول، انتقضت معظم القبائل العربية عدا أهل مكة والطائف والقبائل المجاورة لمكة والمدينة والطائف على خلافة أبي بكر الخليفة الجديد للمسلمين. اختلفت أسباب الانتقاض، فمنهم من ارتد عن الإسلام، ومنهم من ظل على دين الإسلام مع رفضهم أداء فريضة الزكاة، ومنهم من التف حول مدعي النبوة في القبائل العربية.

استغل مانعو الزكاة من قبائل عبس وذبيان وغطفان خروج بعث أسامة بن زيد الذي كان قد أوصى به الرسول قبل وفاته، وحاولوا مهاجمة المدينة. وبعد أن استطاع الخليفة أبو بكر الصديق صد الهجوم، وإرساله من يطارد فلول المنهزمين، عقد أبو بكر أحد عشر لواءً لمحاربة المرتدين ومانعي الزكاة في جميع أرجاء جزيرة العرب. أمّر أبو بكر خالد بن الوليد على أحد تلك الجيوش قوامه 4,000 مقاتل، ووجهه إلى إخضاع طيئ ثم محاربة مدعي النبوة طليحة بن خويلد وقبيلته بني أسد، ثم التوجه لإخضاع بني تميم. إلا أنه وقبل أن يتحرك الجيش، وصل عدي بن حاتم الطائي بأموال زكاة طيء، لتنضم بذلك طيء لجيش خالد.


خط سير خالد بن الوليد في حروب الردة.
اجتمعت قبائل أسد وفزارة وسليم وفلول عبس وذبيان وبكر حول طليحة بن خويلد الذي ادعى النبوة. توجه إليهم خالد بجيشه، واشتبك معهم في بُزاخة، وهزمهم وفرّ طليحة إلى الشام. أمر خالد بعد ذلك بمطاردة فلول المنهزمين، ثم أمر بإحراق الأسرى بالنيران ونكّل بهم، وأرسل رؤساءهم مكبّلين بالأصفاد إلى الخليفة لينظر ماذا يفعل بهم، لما ألحقوا بمن بقوا على دينهم من أذى، وليكون ذلك ردعًا لمن سيلقاه بعد ذلك.

التفت الفلول حول أم زمل التي كانت لها ثارات عند المسلمين، فقد قتل زيد بن حارثة أمها أم قرفة في سريته إلى بني فزارة، لتحريضها قومها على قتال المسلمين. فقاتلهم خالد في معركة كبيرة في ظفر، وهزمهم وقتل أم زمل.

بنو تميم
توجه خالد بعد ذلك بجيشه إلى بني تميم. لم تكن بنو تميم على موقف واحد، فمنهم بطون إيتاء الزكاة وإتباع خليفة رسول الله، ومنهم من رأى عكس ذلك، وبقي فريق ثالث في حيرة من أمرهم. فلما وصل جيش خالد البطاح وهي منزل بنو يربوع، لم يجد بها أحدًا. كان سيدهم مالك بن نويرة ممن كانوا تحيروا في أمرهم، وكان قد أمر قومه بأن يتفرقوا. بثّ خالد السرايا، وأمرهم بأن يأتوه بكل من لم يجب داعية الإسلام، وإن امتنع أن يقتلوه. وكان قد أوصاهم أبو بكر أن يؤذّنوا إذا نزلوا منزلاً، فإن أذن القوم فكفوا عنهم وإن لم يؤذنوا فاقتلوا، وإن أجابوكم إلى داعية الإسلام فسائلوهم عن الزكاة، فإن أقروا فاقبلوا منهم وإن أبوا فقاتلوهم. عندئذ، جاءه الجند بمالك بن نويرة في جماعة من قومه، اختلفت السرية فيهم، فشهد أبو قتادة الأنصاري أنهم أقاموا الصلاة، وقال آخرون: إنهم لم يؤذنوا ولم يصلوا. تم أسر مالك ابن النويرة وجيئ به إلى خالد، أمر خالد بقتل ابن نويرة، واختلف الرواة في سبب قتل خالد مالكًا، فمنهم من قال أن الأسرى قتلوا لأن الليلة كانت باردة، وقد أمر خالد بأن يدفئوا الأسرى، وكانت تعني في لغة كنانة القتل، فقتلهم الحراس. ومنهم من قال أنه دارت بين خالد ومالك حوارًا استنتج منه خالد أن مالكًا ينكر الزكاة، بعدما قال الأخير: "أنا آتي بالصلاة دون الزكاة"، ليرد خالد ويقول: "أما علمت أن الصلاة والزكاة معاً؟ لا تُقبل واحدة دون الأخرى؟"، فقال له ابن النويرة: "قد كان صاحبكم يقول ذلك" ولم يقل نبي الله أو رسول الله، فرد عليه خالد: "أوما تراه صاحباً لك! والله لقد هممت أن أضرب عنقك بهذه الكلمة"، فقال ابن النويرة: "أوبذلك أمرك صاحبك؟" ليتضح لخالد بأن مالك ابن النويرة مرتد فعلاً، وذلك بتحذيره عدة مرات قبل أن يقوم بقتله بأن يدفع الزكاة وبأن لا يسمي النبي بهذه الصيغة التي تبين ردته عن الإسلام، إلى أن مالك أبى بكلا الأمرين، فقتله ضرار ابن الأزور بأمر من خالد. وفي نفس ليلة مقتل مالك، تزوج من أم تميم ليلى بنت المنهال زوجة مالك على أنها سبية، والسبي كان من الأمور المتعارف عليها في الحروب عند المسلمين والكفار، وهذا الزواج أنكره العديد من الصحابة، حتى أن أبو قتادة ترك الجيش وعاد إلى المدينة بسبب خلاف مع خالد ، مقسمًا ألا يجمعه لواء مع خالد بن الوليد. استنكر الصحابة في المدينة فعل خالد، وأرسل أبو بكر في طلب خالد. كان عمر بن الخطاب ممن أغضبه فعل خالد، حتى أنه طلب من الخليفة أن يعزل هذا الأخير، إلا أن أبا بكر رفض ذلك، قائلاً: "ما كنت لأشيم سيفًا سلّه الله على الكافرين". عنّف أبو بكر خالدًا على فعله، ثم صرفه إلى جيشه، وودي مالكًا وردّ سبي بني يربوع.

مسيلمة الكذاب
Crystal Clear app kdict.png مقالة مفصلة: معركة اليمامة

«يَا مُحمّدَاه» شعار المسلمين الذي جعل ينادي به خالد بن الوليد في معركة اليمامة.
ادعى مسيلمة بن حبيب النبوة، واستطاع أن يجمع حوله أربعين ألفًا من قومه بني حنيفة وغيرهم، ممن أقروا بنبوته. وكان في شهادة "الرجَّال بن عنفوة" الذي كان الرسول قد بعثه مع وفد بني حنيفة، حين وفدوا عليه ليعلنوا إسلامهم في عام الوفود ليعلمهم الدين، بأن محمدًا قد أشركه في النبوة، أكبر الدعم له في إدعائه، مما زاد من خطورة فتنته على المسلمين. لذا، فقد وجّه له أبو بكر لواءً بقيادة عكرمة بن أبي جهل، ثم أردفه بلواء آخر بقيادة شرحبيل بن حسنة.

تسرّع عكرمة في قراره بمواجهة جيش مسيلمة وحده قبل أن يدركه جيش شرحبيل بن حسنة، مما عرّضه لهزيمة نكراء. حين وصل شرحبيل بجيشه، أدرك صعوبة الموقف، لذا أرسل للخليفة ليُعلمه بما كان. حينئذ، كان خالد قد فرغ من أمر بني تميم، فأمره أبو بكر بالتوجه من البطاح إلى اليمامة، لقتال مسيلمة الكذاب متنبي بني حنيفة. حين وصل خالد بجيشه إلى ثنية اليمامة، أدرك جيشه سرية من بني حنيفة، فأمر بقتلهم واستبقى رئيسهم مجاعة بن مرارة، لعله يخلُص منه بما ينفعه، وقيّده بالحديد في خيمته، وجعل على حراسته زوجته أم تميم.

نزل مسيلمة بجيشه في عقرباء على أطراف اليمامة. ثم التقى الجمعان، وكانت الغلبة في البداية لبني حنيفة، فتراجع المسلمون حتى دخلوا فسطاط خالد، وكادوا أن يبطشوا بأم تميم لولا أن أجارها مجاعة بن مرارة، لما وجد منها من حسن معاملة. حينئذ، ثارت الحمية في قلوب المسلمين، فأظهر المهاجرون والأنصار بطولاتٍ قلبت دفة المعركة لصالحهم، فتقهقرت بنو حنيفة يحتمون بحديقة مسوّرة منيعة الجدران تسمى بـ "حديقة الرحمن". أدرك المسلمون أنهم إن لم يسرعوا بالظفر بهم، فقد يطول الحصار، فطلب البراء بن مالك من رفقائه أن يحملوه ليتسوّر الحديقة وتبعه بعض زملائه، واستطاعوا فتح باب الحديقة، وأعمل المسلمون القتل في بني حنيفة، وقتل وحشي بن حرب مسيلمة، مما فتّ في عضد بني حنيفة. ومن يومها، أصبحت الحديقة تسمى "بحديقة الموت".

بعد أن انتهت المعركة تحرك خالد بجيشه، ليفتح حصون اليمامة، وكان خالد قد وثق بمجاعة لإجارته لأم تميم. وكان مجاعة قد أرسل للحصون التي لم يكن بها سوى النساء والأطفال والشيوخ ومن لا يستطيعون القتال بأن يلبسوا الدروع. أقنع مجاعة خالدًا بأن الحصون مملوءة بالرجال، ونظر خالد فوجد جيشه قد أنهكته الحروب، وقتل منه الكثير حتى أنه قُدر قتلى المسلمون يوم اليمامة بمائتين وألف منهم 360 من المهاجرين والأنصار، لذا رأى خالد أن يصالحهم على أن يحتفظ المسلمون بنصف السبي والغنائم. عندئذ طلب منه مجاعة أن يذهب ليعرض على قومه الأمر، ثم عاد زاعمًا بأنهم لم يقبلوا العرض، فخفّضه خالد إلى الربع. وحين دخل المسلمون الحصون، لم يجد المسلمون سوى النساء والأطفال والعجزة، غضب خالد لخداعه، إلا أنه وجدها شجاعة من مجاعة، استطاع بها أن يحفظ بها من بقي من قومه، فأجاز الصلح.

بعد أن تم لخالد النصر، طلب من مجاعة أن يزوجه ابنته، فلبّى مجاعة طلبه. تسبب ذلك في إثارة غضب الخليفة وكبار الصحابة، لأنه لم يختر الوقت المناسب لذلك، فقد كانت المدينة في حالة حزن على فقدانهم لألف ومائتي شهيد بينهم 39 من حفظة القرآن الكريم، وهو ما استدعى جمعهم للقرآن. أرسل أبو بكر لخالد فعنّفه أشد مما عنفه يوم زواجه من أم تميم، فتألم خالد لغضب أبي بكر. بعد اليمامة، انتهت مهمة خالد في حروب الردة، فاتخذ له بيتًا في أحد أودية اليمامة عاش فيه مع زوجتيه.


'''


Khaled

'\nلا يعرف الكثير عن خالد خلال فترة الدعوة للإسلام في مكة. وبعد هجرة الرسول من مكة إلى المدينة المنورة، دارت العديد من المعارك بين المسلمين وقريش. لم يخض خالد غزوة بدر أولى المعارك الكبرى بين الفريقين، والتي وقع فيها شقيقه الوليد أسيرًا في أيدي المسلمين. وذهب خالد وشقيقه هشام لفداء الوليد في يثرب، إلا أنه وبعد فترة قصيرة من فدائه، أسلم الوليد وهرب إلى يثرب.\n\nكانت غزوة أحد أولى معارك خالد في الصراع بين القوتين، والتي تولى فيها قيادة ميمنة القرشيين. لعب خالد دورًا حيويًا لصالح القرشيين، فقد استطاع تحويل دفة المعركة، بعدما استغل خطأ رماة المسلمين، عندما تركوا جبل الرماة لجمع الغنائم بعد تفوق المسلمين في بداية المعركة. انتهز خالد ذلك الخطأ ليلتف حول جبل الرماة ويهاجم بفرسانه مؤخرة جيش المسلمين، مما جعل الدائرة تدور على المسلمين، وتحوّل هزيمة القرشيين إلى نصر.\n\nشارك خالد أيضًا في صفوف الأحزاب في غزوة الخندق، وقد تولى هو وعمرو بن العاص تأمين مؤخرة الجيش في مائتي فارس، خوفًا من أن يتعقبهم المسلمون. كما كان على رأس فرسان قريش الذين أرادوا أن يحولوا بين المسلمين ومكة في غزوة الحديبية.\n\nإس

In [ ]:
nlp('محمد').vector

array([ 0.80145264, -0.5097033 ,  0.613964  ,  1.3754942 ,  0.46684232,
       -0.04464621,  0.8612223 , -1.5921508 , -1.9367356 , -2.0546443 ,
       -3.3741977 ,  3.039746  ,  0.06843983, -3.4611075 , -0.02922482,
       -6.249088  ,  0.13342063,  2.415632  , -4.6120048 , -2.558722  ,
        1.2262403 ,  3.057939  , -1.1268952 ,  0.30093428,  1.9931322 ,
       -0.62817836, -1.0539984 ,  2.097052  ,  2.7087996 ,  0.46032926,
        3.4975073 , -0.31189406,  1.4738934 ,  0.63599867,  0.09197995,
       -0.76370686, -1.1170248 ,  0.7429148 ,  1.2113057 , -0.07190861,
        0.12950698,  1.620404  ,  2.2386227 ,  0.20559973, -0.6026682 ,
        2.1381128 ,  2.81794   , -1.5340971 ,  2.3036737 , -3.3349545 ,
       -1.562853  ,  0.14608033, -0.99827707,  1.0818022 ,  0.12351388,
        1.5101413 , -0.95441777, -1.0185006 ,  0.26721156, -0.5419489 ,
        0.3967836 ,  0.47026438,  2.410463  ,  1.5374434 , -2.978941  ,
       -2.8348966 , -0.4507167 ,  1.6771367 , -1.4268402 , -1.03

In [ ]:
nlp('نبياثبتنمراشكمنرلاشب').vector

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [ ]:
def SimilarWords(Word,Text) :
    if Word in Text.split() : 
        WordsDifference = {}
        for EachWord in Text.split() : 
            WordsDifference[EachWord] = sum([abs(float(i)-float(j)) for i,j in zip(nlp(Word).vector,nlp(EachWord).vector)])
        return list({k: v for k, v in sorted(WordsDifference.items(), key=lambda item: item[1])}.items())[:10]
    else : 
        return 'Word not exists'

In [ ]:
SimilarWords('جيش',Khaled)

[('جيش', 0.0),
 ('جيشه', 96.08196453750134),
 ('جيشه،', 96.08196453750134),
 ('أسد', 115.67116642626934),
 ('أسد،', 115.67116642626934),
 ('لواءً', 118.19451783969998),
 ('لواء', 118.19451783969998),
 ('شِعب', 119.81134903430939),
 ('لجيش', 121.00337676145136),
 ('ببسالة', 122.33787403255701)]

In [ ]:
SimilarWords('المعارك',Khaled)

[('المعارك', 0.0),
 ('المعركة،', 88.74795682588592),
 ('المعركة.', 88.74795682588592),
 ('المعركة', 88.74795682588592),
 ('معارك', 96.47501111868769),
 ('الحروب', 106.87993907555938),
 ('الحروب،', 106.87993907555938),
 ('القتال', 113.93580467440188),
 ('معركة', 116.99526475463063),
 ('الدائرة', 132.5453249514103)]

In [ ]:
SimilarWords('مكة',Khaled)

[('مكة.', 0.0),
 ('مكة', 0.0),
 ('مكة،', 0.0),
 ('المدينة', 92.32880971394479),
 ('المدينة.', 92.32880971394479),
 ('تبوك', 94.6174785438925),
 ('ومكة', 103.05900430120528),
 ('لمكة', 109.59293326176703),
 ('المنورة،', 111.00012876465917),
 ('والطائف', 112.02342218160629)]

In [ ]:
egypt = nlp("مصر")
tunisia = nlp("تونس")
apple = nlp("تفاح")

print("egypt Vs. tunisia = ", egypt.similarity(tunisia))
print("egypt Vs. apple = ", egypt.similarity(apple))

egypt Vs. tunisia =  0.8277813628455393
egypt Vs. apple =  0.09689554003631644


In [ ]:
def SimilarWords(Word,Text) :
    if Word in Text.split() : 
        WordsSimilarity = {}
        for EachWord in Text.split() : 
            WordsSimilarity[EachWord] = nlp(Word).similarity(nlp(EachWord))
        return list({k: v for k, v in sorted(WordsSimilarity.items(), key=lambda item: item[1],reverse=True)}.items())[:10]
    else : 
        return 'Word not exists'

In [ ]:
SimilarWords('جيش',Khaled)

[('جيش', 1.0),
 ('جيشه', 0.7570608096849553),
 ('جيشه،', 0.7570608096849553),
 ('شِعب', 0.705097814309367),
 ('لواءً', 0.6995044749596266),
 ('لواء', 0.6995044749596266),
 ('الجيش', 0.6845334657559737),
 ('الجيش،', 0.6845334657559737),
 ('أسد', 0.6307381818590087),
 ('أسد،', 0.6307381818590087)]

In [ ]:
SimilarWords('المعارك',Khaled)

[('المعارك', 1.0),
 ('المعركة،', 0.8371356787953966),
 ('المعركة.', 0.8371356787953966),
 ('المعركة', 0.8371356787953966),
 ('معارك', 0.7988206228120315),
 ('الحروب', 0.7677075872311394),
 ('الحروب،', 0.7677075872311394),
 ('معركة', 0.7380255184970519),
 ('القتال', 0.7178694073944786),
 ('حرب', 0.6241838691886463)]

In [ ]:
SimilarWords('مكة',Khaled)

[('مكة.', 1.0),
 ('مكة', 1.0),
 ('مكة،', 1.0),
 ('المدينة', 0.7335405945594877),
 ('المدينة.', 0.7335405945594877),
 ('تبوك', 0.6869184345211938),
 ('لمكة', 0.6556921641196363),
 ('ومكة', 0.643202455748766),
 ('المنورة،', 0.5809167414956508),
 ('الشام.', 0.5641731261026883)]